In [32]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import seaborn as sns

module_path = (Path().resolve().parent/ "Modules")
sys.path.append(str(module_path))

pd.set_option("display.max_columns", None)

import my_modules, model_tuner, features # 自作モジュール

In [33]:
df = pd.read_csv("../Data/train_data_tmp.csv", encoding="shift-jis")
odds_df = pd.read_csv("../Data/Time_Series_Odds_win_odds.csv", encoding="shift-jis")

In [34]:
df = my_modules.preprocessing(df)
df = my_modules.common_process(df)

C:\Users\ken05\Documents\others\HORSE_RACING\Modules\my_modules.py:74: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["place_num"] = df["place"].replace(place_dict).astype(int)


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 252411 entries, 252634 to 0
Data columns (total 67 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   race_id              252411 non-null  int64         
 1   year                 252411 non-null  int64         
 2   month                252411 non-null  int64         
 3   day                  252411 non-null  int64         
 4   times                252411 non-null  int64         
 5   place                252411 non-null  object        
 6   daily                252411 non-null  object        
 7   race_num             252411 non-null  int64         
 8   horse                252411 non-null  object        
 9   jockey_id            252411 non-null  object        
 10  trainer_id           252411 non-null  int64         
 11  horse_N              252411 non-null  object        
 12  waku_num             252411 non-null  int64         
 13  horse_num          

### リークとなり得る情報から特徴量を作成

In [36]:
def calc_mean_race_development(df_to_copy, feature_col_to_copy, target_col=None, grouping_col=None, feature_name=None):
    if (target_col is None) or (grouping_col is None) or (feature_name is None):
        raise ValueError("target_col, grouping_col and prefix must be selected")
    
    df = df_to_copy.copy()
    feature_col = feature_col_to_copy.copy() 

    grouped1 = df.groupby(grouping_col, observed=True)
    grouped2 = df.groupby(["id_for_fold", *grouping_col], observed=True)

    # 同じ条件で1着になるの確率を計算
    bunsi1 = grouped1[target_col].cumsum() - grouped2[target_col].cumsum()
    bunbo1 = grouped1[target_col].cumcount() - grouped2[target_col].cumcount()

    df[feature_name] = bunsi1 / bunbo1.replace(0, np.nan) 
    feature_col.append(feature_name)

    return df, feature_col

In [37]:
tmp, feature_col = calc_mean_race_development(df, [], target_col="rank", grouping_col=["horse", "dist_type"], feature_name="past_rank_mean")

In [38]:
feature_col

['past_rank_mean']

In [39]:
tmp[tmp.horse == "イクイノックス"][["year", "month", "day", "horse", "dist_type", "rank", "past_rank_mean"]]

,year,month,day,horse,dist_type,rank,past_rank_mean
175554,2021,8,28,イクイノックス,mile,1,NaN
165104,2021,11,20,イクイノックス,mile,1,1.000000
145885,2022,4,17,イクイノックス,intermediate,2,NaN
140241,2022,5,29,イクイノックス,long,2,NaN
121687,2022,10,30,イクイノックス,intermediate,1,2.000000
114337,2022,12,25,イクイノックス,long,1,2.000000
90733,2023,6,25,イクイノックス,long,1,1.500000
75446,2023,10,29,イクイノックス,intermediate,1,1.500000
71709,2023,11,26,イクイノックス,long,1,1.333333


In [20]:
df_tmp = df.time_diff.replace("----", np.nan)
df_tmp.astype(np.float32)

252634    2.0
252635    2.1
252633    1.6
252632    1.6
252631    1.5
         ... 
7         2.0
9         3.4
10        NaN
5         1.6
0        -0.2
Name: time_diff, Length: 252411, dtype: float32